# Tutorial 4:

In [ ]:
# Load packages:

# this package allows to work efficiently with arrays
import numpy as np
# this package is used to draw graphs
import matplotlib.pyplot as plt

---

## Solving tridiagonal systems

In this section, we want to construct an algorithm to solve a linear problem $TV = b$ where the matrix $T$ is invertible, symmetric and tridiagonal, i.e. $T_{i,j} = T_{j,i}$ and $T_{i,j} = 0$ if $|i-j|>1$, or equivalently it is of the form 

$$ T = \left(\begin{array}{ccccc}
\alpha_{1} & \beta_{1} & 0       & \dots  &  0 \\ 
\beta_{1} & \alpha_{2} & \beta_{2} & \ddots & \vdots \\
0 & \ddots & \ddots & \ddots & 0 \\ 
\vdots & \ddots & \beta_{N-2} & \alpha_{N-1} & \beta_{N-1} \\
0 & \dots & 0 & \beta_{N-1} & \alpha_{N}
\end{array}\right).$$


Let us construct a Cholesky decomposition for such matrices. We will suppose that no forbidden operations (division by zero and square roots of negative value) occur.   

1) a) Compute the coefficients of the first column of the matrix $L$ of the Cholesky decomposition $T = LL^T$ as functions of $\alpha_1$ and $\beta_1$. What do you observe? 

b) Compute the coefficients of the second column using those values and $\alpha_2$ and $\beta_2$. What do you observe again? 

c) Asuming that this property holds again for all $i$ first column, prove that it holds again for the $i+1$-th. 

d) Let us denote $\gamma = (\gamma_i)_{i=1,\dots,N}$ and $\kappa=(\kappa_i)_{i=1,\dots,N-1}$ the diagonal and subdiagonal terms of $L$. Write down the iterative sequence satisfied by $(\gamma_i,\kappa_i)$, i.e. write $(\gamma_{i+1},\kappa_{i+1})$ as a function of the previously computed values $(\gamma_i,\kappa_i)$ and on the $(\alpha_i,\beta_i)$. 

**Answer:**

a) $$L_{1,1} = \sqrt{\alpha_1}, \qquad L_{2,1} = \frac{\beta_1}{L_{1,1}} = \frac{\beta_1}{\sqrt{\alpha_1}}, \qquad L_{i,1} = 0 \quad\text{for all}\quad i>2.$$

The first column possesses the same zeros as the first column of $T$.

b)  $$L_{2,2} = \sqrt{\alpha_2-L_{2,1}^2} = \sqrt{\alpha_2-\frac{\beta_1^2}{\alpha_1}}, \qquad L_{3,2} = \frac{\beta_2}{L_{2,2}} = \frac{\beta_2}{\sqrt{\alpha_2-\frac{\beta_1^2}{\alpha_1}}}, \qquad L_{i,2} = 0 \quad\text{for all}\quad i>3.$$

The second column possesses the same zeros as the first column of $T$.

c) If all the columns until $i$-th column are composed of two terms, one $L_{j,j}$ on the diagonal and one $L_{j+1,j}$ on the subdiagonal, then 

$$\alpha_{i+1} = L_{i+1,i+1}^2 + L_{i+1,i}^2 \quad \Rightarrow \quad L_{i+1,i+1} = \sqrt{\alpha_{i+1}- L_{i+1,i}^2}$$

and the subdiagonal terms read 

$$ \beta_{i+1} = L_{i+1,i+1} L_{i+2,i+1} + 0 + 0 + ... \quad \Rightarrow \quad L_{i+2,i+1} = \frac{\beta_{i+1}}{L_{i+1,i+1}}$$
$$ 0 = L_{i+1,i+1} L_{i+1,j} \quad \text{for} \quad j>i+1\quad\Rightarrow\quad L_{i+1,j} = 0 \quad \text{for} \quad j>i+1$$

Then the Cholesky decomposition only possesses a diagonal and a subdiagonal. 

d) $$\gamma_{i+1} = L_{i+1,i+1} = \sqrt{\alpha_{i+1} - L_{i+1,i}^2} = \sqrt{\alpha_{i+1} - \kappa_i^2},\qquad \kappa_{i+1} = L_{i+2,i+1} = \frac{\beta_{i+1}}{L_{i+1,i+1}} = \frac{\beta_{i+1}}{\gamma_{i+1}} = \frac{\beta_{i+1}}{ \sqrt{\alpha_{i+1} - \kappa_i^2} }  $$

Now, we will exploit this property to implement the Cholesky algorithm without performing the multiplications and additions by zeros.  

In practice, only the non-zero coefficients of $T$ are stored, and only once, i.e. $T$ is represented by the two vectors $\alpha = (\alpha_i)_{i=1,\dots,N}$ and $\beta=(\beta_i)_{i=1,\dots,N-1}$.

Similarly, only the non-zero coefficients of $L$ are stored, i.e. the two vectors $\gamma = (\gamma_i)_{i=1,\dots,N}$ and $\kappa=(\kappa_i)_{i=1,\dots,N-1}$. 

2) a) In the test below, we will use the matrix 

$$T = \left( \begin{array}{ccc} 1 & -1 & 0 \\ -1 & 5 & 6 \\ 0 & 6 & 10\end{array} \right). $$

Compute the Cholesky decomposition of this matrix. 

b) Implement a function that returns the Cholesky decomposition in this format. It should take $\alpha$ and $\beta$ as arguments and return $\gamma$ and $\kappa$.  

c) Test your algorithm with the matrix given in a) and verify your result.

**Answer:**

a) $$L = \left( \begin{array}{ccc} 1 & 0 & 0 \\ -1 & 2 & 0 \\ 0 & 3 & 1 \end{array}\right)$$

In [ ]:
#b)
def Cholesky_Tridiag(alpha,beta):
    """
    Compute the Cholesky decomposition of a tridiagonal matrix
    ----------   
    parameters:
    alpha : vector of diagonal coefficients of T (numpy array of size N)
    beta  : vector of subdiagonal coefficients of T (numpy array of size N-1)
    
    returns:
    gamma : vector of diagonal coefficients of L (numpy array of size N)
    kappa : vector of subdiagonal coefficients of L (numpy array of size N-1)
    """
    
    gamma = np.copy(alpha)
    kappa = np.copy(beta)
    
    gamma[0] = np.sqrt(alpha[0])
    for i in range(len(beta)):
        if(gamma[i] == 0): 
            print("Error division by zero in Cholesky tridiagonal")
            return
        else:
            kappa[i]   = beta[i]/gamma[i]
        
        res = alpha[i+1]-kappa[i]**2
        if(res < 0): 
            print("Error sqrt of negative value in Cholesky tridiagonal")
            return
        else:
            gamma[i+1] = np.sqrt(alpha[i+1]-kappa[i]**2) 
    
    return gamma, kappa

In [ ]:
#c)
alpha = np.array([1,5,10])
beta  = np.array([-1,6])

gamma, kappa = Cholesky_Tridiag(alpha,beta)
print('gamma =', gamma)
print('kappa =', kappa)

Similarly, we will exploit this format to solve a tridiagonal linear system. 
Consider a lower triangular matrix $L$ which only possesses a non-zero diagonal $\gamma$ and subdiagonal $\kappa$. 

3) a) Write down every component $V_i$ of the solution of the problem $LV = b$ as a function of $\gamma$ and $\kappa$ and $b$ as given by the forward substitution algorithm.  

b) Write down every component $V_i$ of the solution of the problem $L^T V = b$ as a function of $\gamma$ and $\kappa$ and $b$ as given by the back substitution algorithm.  

**Answer:** 

a) $$V_1 = \frac{b_1}{\gamma_1}, \qquad V_i = \frac{b_i - \kappa_{i-1} V_{i-1}}{\gamma_i}$$

b) $$V_N = \frac{b_N}{\gamma_N}, \qquad V_i = \frac{b_i - \kappa_{i} V_{i+1}}{\gamma_i}$$ 

Be carefull about the indices in the back substitution! 

4) a) In the test below, we will use the parameters

$$L = \left( \begin{array}{ccc} 1 & 0 & 0 \\ -1 & 2 & 0 \\ 0 & 3 & 1 \end{array}\right), \qquad b = \left(\begin{array}{ccc} 1 \\ 2 \\ 3 \end{array}\right).$$

Compute the solutions of $LV = b$ and $L^T V = b$

b) Implement a function that solves a tridiagonal lower triangular system using the tridiagonal format. It should take $\gamma$, $\kappa$ and $b$ as arguments and return $V$.  

c) Implement a function that solves a tridiagonal lower triangular system using the tridiagonal format. It should take $\gamma$, $\kappa$ and $b$ as arguments and return $V$.  

d) Test both of them with the parameters in a) and verify your results. 

**Answer:**

a) $V = (1, \frac{3}{2}, -\frac{3}{2})^T, \qquad V = (-\frac{5}{2}, -\frac{7}{2}, 3)^T$

In [ ]:
#b)
def forward_Tridiag(gamma,kappa,b):
    """
    Compute the solution of a lower triangular tridiagonal system LV=b
    ----------   
    parameters:
    gamma : vector of diagonal coefficients of L (numpy array of size N)
    kappa : vector of subdiagonal coefficients of L (numpy array of size N-1)
    b     : RHS of the problem LV = b (numpy array of size N)
    
    returns:
    V     : solution of the problem LV = b (numpy array of size N)
    """
    
    if(any(gamma<0)):
        print("Non invertible matrix in tridiagonal forward substitution")
        return
    
    V = np.copy(gamma)
    
    V[0] = b[0]/gamma[0]
    for i in range(len(b)-1):
        V[i+1] = (b[i+1]-kappa[i]*V[i])/gamma[i+1]
    
    return V

In [ ]:
#c)
def back_Tridiag(gamma,kappa,b):
    """
    Compute the solution of a lower triangular tridiagonal system L^T V = b
    ----------   
    parameters:
    gamma : vector of diagonal coefficients of L (numpy array of size N)
    kappa : vector of subdiagonal coefficients of L (numpy array of size N-1)
    b     : RHS of the problem L^T V = b (numpy array of size N)
    
    returns:
    V     : solution of the problem L^T V = b (numpy array of size N)
    """
    
    if(any(gamma<0)):
        print("Non invertible matrix in tridiagonal forward substitution")
        return
    
    N = len(b)
    V = np.zeros(N)
    
    j = N-1
    V[j] = b[j]/gamma[j]
    for i in range(N-1):
        j -= 1
        V[j] = (b[j]-kappa[j]*V[j+1])/gamma[j]
    
    return V

In [ ]:
#d)
gamma = np.array([1.,2.,1.])
kappa = np.array([-1.,3.])
b     = np.array([1.,2.,3.])

V = forward_Tridiag(gamma,kappa,b)
print('forward: V =', V)

V = back_Tridiag(gamma,kappa,b)
print('back: V =', V)

5) a) In the test below, we will use the parameters of 2)a). 
Compute the solutions of $T V = b$ with $b = (1,2,3)^T$. 

b) Implement a function that that solves the problem $L L^T V = b$ where $L$ is a tridiagonal lower triangular matrix (given by a Cholesky decomposition). 

c) Test it with the parameters in a) and verify your computations. 

**Answer:** 

a) $V = (4, 3, -\frac{3}{2})^T$ 

In [ ]:
#b)
def solve_tridiag(gamma,kappa,b):
    """
    Compute the solution of a lower triangular tridiagonal system L L^T V = b
    ----------   
    parameters:
    gamma : vector of diagonal coefficients of L (numpy array of size N)
    kappa : vector of subdiagonal coefficients of L (numpy array of size N-1)
    b     : RHS of the problem L^T V = b (numpy array of size N)
    
    returns:
    V     : solution of the problem L^T V = b (numpy array of size N)
    """
    W = forward_Tridiag(gamma,kappa,b)
    return back_Tridiag(gamma,kappa,W)

In [ ]:
#c)
gamma = np.array([1.,2.,1.])
kappa = np.array([-1.,3.])
b     = np.array([1.,2.,3.])

V = solve_tridiag(gamma,kappa,b)
print("V=",V)

## Application: a Legendre like equation

Consider a 2nd order differential equation of the form 

$$\frac{d}{dx}\left( (1-x^2) \frac{d}{dx} f(x) \right) - f(x) = S(x), \quad \forall x\in]-1,+1[,$$

for some source term $S(x)$.
We consider a discretization of this equation under the form 

$$ \frac{(1-x_{i+1/2}^2) \frac{V_{i+1}-V_i}{h} - (1-x_{i-1/2}^2) \frac{V_{i}-V_{i-1}}{h} }{2h} - V_i = S_i, \quad \forall i=1,\dots,N,$$

where $V_i$ is supposed to approximate $f(x_i)$, and we have fixed $h = \frac{2}{N}$ and $x_i = -1 + (i-1/2) h$ and $x_{i+1/2} = -1 + ih$ such that $x_{1-1/2} = x_{1/2} = -1$ and $x_{N+1/2} = 1$. 

7) Write the linear system satisfied by the vector $V = (V_i)_{i=1,\dots,N}$ under the form $AV = b$. Show that the matrix $A$ is symmetric tridiagonal and give its components as function of $x_{i\pm1/2}$, $h$ and $K$. Using this notation, $b_i = S_i$.

**Answer:** 

$$A_{i,i} = -\frac{1-x_{i+1/2}^2}{2h^2} - \frac{1-x_{i-1/2}^2}{2h^2} -1 = -1 +  \left(\frac{-2+x_{i+1/2}^2+x_{i-1/2}^2}{2h^2}\right), \qquad A_{i,i\pm 1} = \frac{1-x_{i\pm 1/2}^2}{2h^2},\qquad A_{i,i\pm k} = 0 \quad\text{for}\quad k>1$$ 

One remarks that $A_{i,i+1} = \frac{1-x_{i+1/2}^2}{2h^2} = \frac{1-x_{i+1-1/2}^2}{2h^2} = A_{i+1,i+1-1} = A_{i,+1,i}$ is indeed symmetric.

8) For the test below, we fix $N = 100$ and $S_i = S(x_i) = 10 \exp(-10 x_i^2)$. 

a) Implement a function that constructs the matrix $A$ in the tridiagonal format of the last part. 

b) One can prove that the matrix $A$ is symmetric negative definite (this is admitted here). Then, we will solve $(-A)V = -b$ as the matrix $-A$ is symmetric positive definite. Use the functions of the previous to solve this system $(-A)V = -b$.

c) Plot the graph $(V_i,x_i)_{i=1,\dots,N}$ which is suppose to approximate the function $f$. 

In [ ]:
# a)
def construct_matrix(N):
    """
    Construct the matrix A of size NxN in the tridiagonal format
    ----------   
    parameters:
    N     : size of the matrix 
    
    returns:
    alpha : diagonal of the considered tridiagonal matrix (numpy array of size N)
    beta  : subdiagonal of the considered tridiagonal matrix (numpy array of size N)
    """
    alpha = np.zeros(N)
    beta  = np.zeros(N-1)
    
    h        = 2./N
    xip12    = -1.
    for i in range(N-1):
        xim12  = xip12
        xip12 += h
        alpha[i] = -(1.-xip12**2)/(2*h**2) - (1.-xim12**2)/(2*h**2) - 1.
        beta[i]  =  (1.-xip12**2)/(2*h**2)
    xim12  = xip12
    xip12 += h
    alpha[i+1] = -(1.-xip12**2)/(2*h**2) - (1.-xim12**2)/(2*h**2) - 1.

    return alpha, beta

In [ ]:
#b) 
N            = 100
alpha, beta  = construct_matrix(N)
gamma, kappa = Cholesky_Tridiag(-alpha,-beta)

xi = np.linspace(0,N,N)*2./N-1.
b  = np.exp(-xi**2)

V = solve_tridiag(gamma,kappa,b)

In [ ]:
#c)
plt.figure()
plt.plot(xi,V, color='blue', label="f(x)")
plt.xlabel("x")
plt.ylabel("f")
plt.grid()
plt.legend()
plt.show()